In [ ]:
# MEPS Weather Maps
# Region: Northern Europe, focused on Finland
# Products: Surface, Pressure, Severe, Winter

# Data Imports
from siphon.catalog import TDSCatalog
import xarray as xr
import metpy
import cfgrib
import eccodes
from datetime import datetime, timedelta, date, time
from netCDF4 import num2date
import pandas as pd
import geopandas as gpd
import metpy.calc as mpcalc
import metpy.plots as mpplots
from metpy.units import units

# Plot Imports
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

In [ ]:
# ACCESS ROAD NETWORK DATA
roadshp = gpd.read_file('C:tieosoiteverkko_0/tieosoiteverkkoLine.shp')

In [ ]:
# Model Run Selection
# Available runs can be found here: https://thredds.met.no/thredds/catalog/mepslatest/catalog.html
# File names have this format: meps_det_2_5km_YYYYMMDDTHHZ.ncml
Idate = input('Enter Initialization Time: YYYYMMDD')
Itime = input('Enter Initialization Time: HH')
meps_data = TDSCatalog('https://thredds.met.no/thredds/catalog/mepslatest/catalog.xml?dataset=mepslatest/meps_det_2_5km_'+Idate+'T'+Itime+'Z.ncml')
meps_ds = meps_data.datasets[0]
ds = xr.open_dataset(meps_ds.access_urls['OPENDAP'])

In [ ]:
#SELECT PRODUCT CATEGORY    
Category = input('Select Category: a) Sfc b) Pressure c) Severe d) Winter')

# Collect universal data
time_ds = ds.metpy.parse_cf('time')
reftime = ds.metpy.parse_cf('forecast_reference_time')
projectionds = ds.metpy.parse_cf('projection_lambert')
x_crd = ds.metpy.parse_cf('x')
y_crd = ds.metpy.parse_cf('y')
clons, clats = np.meshgrid(x_crd, y_crd)
latnum = clats.shape[0]
lonnum = clons.shape[0]
hmsl = ds.metpy.parse_cf('height_above_msl')
FF00 = reftime

# SURFACE
if Category == 'a': 
    Product = input('Select Product: a) Temperature b) Dew Point c) Wind d) Precipitation e) IR')
   
    if Product == 'a':      # Temperature
        tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
        uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
        vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')

    elif Product == 'b':    # Dew Point
        rh_2m_ds = ds.metpy.parse_cf('relative_humidity_2m')
        tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
        uwnd_10m_ds = ds.metpy.parse_cf('x_wind_10m')
        vwnd_10m_ds = ds.metpy.parse_cf('y_wind_10m')

    elif Product == 'c':    # Wind
        uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
        vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
        wnd_gust_ds = ds.metpy.parse_cf('wind_speed_of_gust')
        wnd_speed_ds = ds.metpy.parse_cf('wind_speed')

    elif Product == 'd':    # Precipitation
        Ptime = input('Select Precipitation timeframe: a) Instantaneous b) 1hr')
        
        if Ptime == 'a':    # Instantaneous Precipitation Type
            rainfall_amount_ds = ds.metpy.parse_cf('rainfall_amount')
            snowfall_amount_ds = ds.metpy.parse_cf('snowfall_amount')
            graupel_amount_ds = ds.metpy.parse_cf('graupelfall_amount')
            MSLP_ds = ds.metpy.parse_cf('air_pressure_at_sea_level')
            uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
            vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
            
        if Ptime == 'b':    # 1hr Precipitation
            precip_acc_ds = ds.metpy.parse_cf('precipitation_amount_acc')
            MSLP_ds = ds.metpy.parse_cf('air_pressure_at_sea_level')
            precipitation_type_ds = ds.metpy.parse_cf('precipitation_type')
            snowfall_amount_over_time_ds = ds.metpy.parse_cf('integral_of_snowfall_amount_wrt_time')
            snowfall_amount_acc_ds = ds.metpy.parse_cf('snowfall_amount_acc')

    elif Product == 'e':    # IR Satellite
        IR_ds = ds.metpy.parse_cf('toa_outgoing_longwave_flux')

    else:
        print('Invalid Parameter Selected')
        

# PRESSURE
if Category == 'b': 
    Level = input('Enter Pressure Level: a) 1000mb b) 925mb c) 850mb d) 700mb e)500mb f)300mb')
    Product = input('Select Product: a) Wind b) Temperature c) Relative Humidity d) Dewpoint e) Vertical Velocity')
    if Product == 'a':                              # Wind
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')
        geop_pl_ds = ds.metpy.parse_cf('geopotential_pl')
    if Product == 'c' or Product == 'd':            # Relative Humidity and Dew Point
        rh_pl_ds = ds.metpy.parse_cf('relative_humidity_pl')
        geop_pl_ds = ds.metpy.parse_cf('geopotential_pl')
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl') 
    if Product == 'e':                              # Vertical Velocity
        w_pl_ds = ds.metpy.parse_cf('upward_air_velocity_pl')
        geop_pl_ds = ds.metpy.parse_cf('geopotential_pl')
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')

    if Level == 'a':
        plevel = 1000       # Pressure level (hPa)
        lgp_min = 4         # Minimum geopotential height (dam)
        lgp_max = 120       # Maximum geopotential height (dam)
    elif Level == 'b':
        plevel = 925
        lgp_min = 4
        lgp_max = 200
    elif Level == 'c':
        plevel = 850
        lgp_min = 100
        lgp_max = 220
    elif Level =='d':
        plevel = 700
        lgp_min = 4
        lgp_max = 400
    elif Level == 'e':
        plevel = 500
        lgp_min = 480
        lgp_max = 600
    elif Level == 'f':
        plevel = 300
        lgp_min = 4
        lgp_max = 1200
    else: 
        print('Invalid Pressure Level')

# SEVERE WEATHER
if Category == 'c':
    option = input("a) Single Parameter b) Composite")
    if option == 'a': 
        Product = input("a) MUCAPE b) CIN c) LCL d) LFC e) PBL Thickness f) PW g) Lightning h) Cloud Tops i) Reflectivity j) 700-500 Lapse Rate")
        if Product == 'a':      # MUCAPE
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
        if Product == 'b':      # CIN
            cin_ds = ds.metpy.parse_cf('atmosphere_convective_inhibition')
        if Product == 'c':      # LCL
            lcl_ds = ds.metpy.parse_cf('lifting_condensation_level')
        if Product == 'd':      # LFC
            lfc_ds = ds.metpy.parse_cf('atmosphere_level_of_free_convection')
        if Product == 'e':      # PBL
            pbl_ds = ds.metpy.parse_cf('atmosphere_boundary_layer_thickness')
        if Product == 'f':      # PW
            pw_ds = ds.metpy.parse_cf('lwe_thickness_of_atmosphere_mass_content_of_water_vapor')
        if Product == 'g':      # Lightning
            ltg_ds = ds.metpy.parse_cf('lightning_index')
        if Product == 'h':      # Cloud Tops
            cloud_top_ds = ds.metpy.parse_cf('cloud_top_altitude')
        if Product == 'i':      # Reflectivity
            rain_ds = ds.metpy.parse_cf('rainfall_amount')
        if Product == 'j':      # Lapse Rates
            tmp_pl_ds = ds.metpy.parse_cf('air_temperature_pl')
            geop_ds = ds.metpy.parse_cf('geopotential_pl')
    if option == 'b':
        Product = input("a) T/Wind/MUCAPE b) Sfc Vorticity & MUCAPE c) MUCAPE and 850mb Winds")
        if Product == 'a':      # Temperature, Wind, MUCAPE
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
            uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
            vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
        if  Product == 'b':     # MUCAPE, 10m Vorticity
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
            vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
        if Product == 'c':      # MUCAPE, 850mb Wind
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
            vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')

# WINTER
if Category == 'd':
    Product = input('Select Product: a) Total Snowfall 10:1')
        # b, c, d provide snowfall over past few hours, still needs work
    if Product == 'a' or Product == 'b' or Product == 'c' or Product == 'd':
        snowfall_amount_time_integral_ds = ds.metpy.parse_cf('integral_of_snowfall_amount_wrt_time')
    if Product == 'e' :
        ptype_ds = ds.metpy.parse_cf('precipitation_type')
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')
        
# Required for pressure products
tmp_pl_ds = ds.metpy.parse_cf('air_temperature_pl')
tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')

In [ ]:
# SELECT VALID TIME
Year    = input('Valid YYYY')
Month   = input('Valid MM')
Day     = input('Valid DD')
Time    = input('Valid HH')

dt = str(Year)+'-'+str(Month)+'-'+str(Day)+'T'+str(Time)+':00:00.000000000'
dt_1 = str(Year)+'-'+str(Month)+'-'+str(Day)+'T'+str(Time)+':00:00'
dt_1

In [ ]:
# SELECT MODEL DATA AT VALID TIME
valid_time = time_ds.sel(time=dt, method='nearest') 

# SURFACE
if Category == 'a':
    if Product == 'a': # 2m Temperature
        tmp2m = tmp_2m_ds.sel(height1=2.0, time=dt, method='nearest')
        uwnd2m = uwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
        vwnd2m = vwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')

    elif Product == 'b': # 2m Dewpoint
        rh2m = rh_2m_ds.sel(height1 = 2.0, time=dt, method='nearest')
        tmp2m = tmp_2m_ds.sel(height1=2.0, time=dt, method='nearest')
        td2m = mpcalc.dewpoint_from_relative_humidity(tmp2m, rh2m)
        uwnd10m = uwnd_10m_ds.sel(height7=10.0, time=dt, method='nearest')
        vwnd10m = vwnd_10m_ds.sel(height7=10.0, time=dt, method='nearest')

    elif Product == 'c': # 10m Wind
        uwnd2m = uwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
        vwnd2m = vwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
        wndG = wnd_gust_ds.sel(height7=10.0, time=dt, method='nearest')
        wndS = wnd_speed_ds.sel(height7=10.0, time=dt, method='nearest')

    elif Product == 'd': # Precipitation
        mslp = MSLP_ds.sel(time=dt, height_above_msl=0.0, method='nearest')

        if Ptime == 'a': # Instantaneous Precipitation Type
            Rain_Instant1 = rainfall_amount_ds.sel(time=dt,height0=0.0, method='nearest')
            Rain_Instant = Rain_Instant1.where(Rain_Instant1 >= (0.2/3600))
            Snow_Instant1 = snowfall_amount_ds.sel(time=dt,height0=0.0, method='nearest')
            Snow_Instant = Snow_Instant1.where(Snow_Instant1 >= (0.2/2600))
            Graupel_Instant1 = graupel_amount_ds.sel(time=dt, height0=0.0, method='nearest')
            Graupel_Instant = Graupel_Instant1.where(Graupel_Instant1 >= (0.2/3600))
            uwnd2m = uwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
            vwnd2m = vwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
        
        if Ptime == 'b': # 1h Accumulated Precipitation
            index = precip_acc_ds.indexes['time'].get_loc(dt) - 1
            precip1_acm = precip_acc_ds.sel(time=dt, height0=0.0, method='nearest')
            precip0_acm = precip_acc_ds.isel(time=index, height0=0)
            precip_1hr = precip1_acm - precip0_acm

            snow_integral = snowfall_amount_over_time_ds.sel(time=dt, height0=0.0, method='nearest')
            snow_acm = snowfall_amount_acc_ds.sel(time=dt, height0=0.0, method='nearest')
            ptype = precipitation_type_ds.sel(time=dt, method='nearest')

    elif Product == 'e': # Infrared Satellite
        ir1 = IR_ds.sel(top_of_atmosphere=0, time=dt, method='nearest')
        # Calculate Temperature using Stefan Boltzmann Law
        ir = (( abs(ir1) / (5.67*(10**-8)))**(1/4)) - 273.15
        ir_30 = ir.where(ir <= -30)
        ir_3050 = ir_30.where(ir_30 >= -50)
        ir_5070 = ir.where(ir <= -50)

# PRESSURE
elif Category == 'b':
    if Product == 'a':
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        geop_pl = geop_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        swnd_pl = mpcalc.wind_speed(uwnd_pl, vwnd_pl)
    if Product == 'b':
        tmp_pl = tmp_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        geop_pl = geop_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
    if Product == 'c':
        rh_pl = rh_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        geop_pl = geop_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
    if Product == 'd':
        rh_pl = rh_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        tmp_pl = tmp_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        td_pl = mpcalc.dewpoint_from_relative_humidity(tmp_pl, rh_pl)
        geop_pl = geop_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
    if Product == 'e':
        w_pl = w_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        wp_pl = w_pl.where(w_pl > 0)
        wn_pl = w_pl.where(w_pl < 0)
        geop_pl = geop_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, time=dt, method='nearest')
        tmp_pl = tmp_pl_ds.sel(pressure=plevel, time=dt, method='nearest')

    
# SEVERE
elif Category == 'c':
    if option == 'a':
        if Product == 'a':
            cape1 = cape_ds.sel(height0=0.0, time=dt, method='nearest')
            cape = cape1.where((cape1 > 25))
        if Product == 'b':
            cin1 = cin_ds.sel(height0=0.0, time=dt, method='nearest')
            cin = cin1.where((cin1 != 1000.0061)) 
        if Product == 'c':
            lcl = lcl_ds.sel(height0=0.0, time=dt, method='nearest')
        if Product == 'd':
            lfc = lfc_ds.sel(height0=0.0, time=dt, method='nearest')
        if Product == 'e':
            pbl = pbl_ds.sel(height0=0.0, time=dt, method='nearest')
        if Product == 'f':
            pw = pw_ds.sel(surface=0, time=dt, method='nearest')
        if Product =='g':
            ltg = ltg_ds.sel(height0=0.0, time=dt, method='nearest')
        if Product =='h':
            cld_top = cloud_top_ds.sel(surface=0, time=dt, method='nearest')
        if Product == 'i':
            rain_rate = rain_ds.sel(height0=0.0, time=dt, method='nearest')
            rain_rate = rain_rate * 3600
            #dbz = 10log(aR**b)
            dbz_r = 10*np.log10(300*rain_rate**1.5)
        if Product == 'j':
            tmp_pl = tmp_pl_ds.sel(time=dt, method='nearest')
            hght5 = geop_ds.sel(time=dt, pressure=500, method='nearest') / 9.81
            hght7 = geop_ds.sel(time=dt, pressure=700, method='nearest') / 9.81
            d57 = hght5 - hght7 
            lr57 = (tmp_pl.sel(pressure=700.00, method='nearest') - tmp_pl.sel(pressure=500.00, method='nearest')) / (d57/1000)
    if option == 'b':
        if Product == 'a':
            cape = cape_ds.sel(height0=0.0, time=dt, method='nearest')
            tmp2m = tmp_2m_ds.sel(height1=2.0, time=dt, method='nearest')
            uwnd2m = uwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
            vwnd2m = vwnd_2m_ds.sel(height7=10.0, time=dt, method='nearest')
        if Product == 'c':
            cape = cape_ds.sel(height0=0.0, time=dt, method='nearest')
            uwnd_85 = uwnd_pl_ds.sel(pressure=850, time=dt, method='nearest')
            vwnd_85 = vwnd_pl_ds.sel(pressure=850, time=dt, method='nearest')

# WINTER
elif Category == 'd':
    if Product == 'a':
        snow_acm = snowfall_amount_time_integral_ds.sel(time=dt, height0=0.0, method='nearest')
        #snow_acm = snow_acm.where((snow_acm > 1))
    if Product == 'b':
        index = snowfall_amount_time_integral_ds.indexes['time'].get_loc(dt) - 1
        snow_acm1 = snowfall_amount_time_integral_ds.sel(time=dt, height0=0.0, method='nearest')
        snow_acm0 = snowfall_amount_time_integral_ds.isel(time=index, height0=0)
        snow_acm = snow_acm1 - snow_acm0
        snow_acm = snow_acm.where((snow_acm > 0))
    if Product == 'c':
        index = snowfall_amount_time_integral_ds.indexes['time'].get_loc(dt) - 6
        snow_acm1 = snowfall_amount_time_integral_ds.sel(time=dt, height0=0.0, method='nearest')
        snow_acm0 = snowfall_amount_time_integral_ds.isel(time=index, height0=0)
        snow_acm = snow_acm1 - snow_acm0
        snow_acm = snow_acm.where((snow_acm > 0))
    if Product == 'd':
        index = snowfall_amount_time_integral_ds.indexes['time'].get_loc(dt) - 12
        snow_acm1 = snowfall_amount_time_integral_ds.sel(time=dt, height0=0.0, method='nearest')
        snow_acm0 = snowfall_amount_time_integral_ds.isel(time=index, height0=0)
        snow_acm = snow_acm1 - snow_acm0
        snow_acm = snow_acm.where((snow_acm > 0))
    if Product == 'e':
        ptype = ptype_ds.sel(time=dt, height0=0.0, method='nearest')

In [ ]:
# Set up Plot
fig = plt.figure(figsize=(20, 15), dpi=200)
ax = fig.add_subplot(1, 1, 1, projection=ccrs.TransverseMercator(central_longitude=21, false_easting=1500000))

Sector = input('Sector Options: >Full >National >South >North >Custom')

if Sector == 'Full':
    ax.set_extent((1, 36, 52, 74), crs=ccrs.PlateCarree())
    length = 4.5            # Windbarb Length
    res = 25                # Wndbarb spatial resolution
    fraction1 = 0.0154      # Colorbar fraction
    xa = 0.12               # X coordinate of water mark
    ya = 0.97               # Y coordinate o water mark
elif Sector == 'National':
    ax.set_extent((15, 34, 58, 70.5), crs=ccrs.PlateCarree())
    length=5
    fraction1 = 0.0128
    res = 15
    xa = 0.14
    ya = 0.97
elif Sector == 'South':
    ax.set_extent((17.4, 33.4, 59.6, 64.5), crs=ccrs.PlateCarree())
    fraction1 = 0.023
    res = 10
    xa = 0.09
    ya = 0.97
elif Sector == 'North':
    ax.set_extent((17.4, 33.4, 64.5, 70.16), crs=ccrs.PlateCarree())
    fraction1 = 0.01857
    length=5
    res = 15
    xa = 0.11
    ya = 0.97
elif Sector == 'Custom':
    LocLat = 53.6179
    LocLon = 22.2542
    LatN = LocLat + 1.5
    LatS = LocLat - 1.5
    LonE = LocLon + 5
    LonW = LocLon - 5
    ax.set_extent((LonW, LonE, LatS, LatN), crs=ccrs.PlateCarree())
    fraction1 = 0.023
    res = 10
    xa = 0.09
    ya = 0.95

#----------------------------------------#

# Establish Features
lakes = cfeature.NaturalEarthFeature('physical', 'lakes', '10m', edgecolor='k', facecolor='none')
# Plot Features
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.4)
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.add_feature(lakes, linewidth=0.1)

# Establish coordinate system used for road file
globe = ccrs.Globe(ellipse='GRS80')
data_crs = ccrs.TransverseMercator(false_easting=500000, false_northing=0, 
                                   central_longitude=27.0, central_latitude=0, 
                                   scale_factor=0.9996, globe=globe)
if Sector in ('North','South','National','Custom'):
    ax.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='black', linewidths=0.7, alpha=0.88)
    ax.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='gold', linewidths=0.4, alpha=1)
else:
    pass

#----------------------------------------#

# BUILD COLORMAPS
# -----------------------------------------------------------------------------------

# Radar Colormap
# --
color_db0020 = plt.cm.bone_r(np.linspace(0,0.55,73))
#color_db2060 = plt.cm.nipy_spectral(np.linspace(0.4,0.90,146))
color_db2030 = plt.cm.nipy_spectral(np.linspace(0.4,0.7,37))
color_db3050 = plt.cm.nipy_spectral(np.linspace(0.7, 0.9,110))
color_db6070 = plt.cm.gist_ncar(np.linspace(0.85,1,37))
color_db = np.vstack((color_db0020, color_db2030, color_db3050, color_db6070))
cmap_db = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_db)
# --     

# Temperature Colormap
# --
#color_TM2030 = plt.cm.Purples(np.linspace(0,1,43))
color_TM0020 = plt.cm.gist_rainbow_r(np.linspace(0,0.42,128))
#color_TM0020 = plt.cm.Blues(np.linspace(0.2,0.80,85))
color_T0020 = plt.cm.turbo(np.linspace(0.2,1,128))
color_T3040 = plt.cm.Reds(np.linspace(0.5,1,43))
color_T = np.vstack((color_TM0020, color_T0020))
cmap_t = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_T)
# --

# Dewpoint Colormap
# --
color_TdM0030 = plt.cm.BrBG(np.linspace(0,0.45,128))
color_Td0015 = plt.cm.Greens(np.linspace(0.1,1,64))
color_Td1520 = plt.cm.BrBG(np.linspace(0.7,1,21))
color_Td2030 = plt.cm.Purples(np.linspace(0.5,1,43))
color_td = np.vstack((color_TdM0030, color_Td0015, color_Td1520, color_Td2030))
cmap_td = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_td)
# --

# Wind Speed Colormap
# --
color_ws0025 = plt.cm.BuPu(np.linspace(0,0.9,160))
color_ws2540 = plt.cm.Wistia(np.linspace(0.4,1,96))
color_ws = np.vstack((color_ws0025, color_ws2540))
cmap_ws = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_ws)
# --     

# IR Satellite Colormap
# --
colors_ir1 = plt.cm.Greys(np.linspace(0.1,1,128))
colors_ir2 = plt.cm.GnBu(np.linspace(0.1, 0.7, 64))
colors_ir3 = plt.cm.magma(np.linspace(0,1, 64))
colors_ir = np.vstack((colors_ir3, colors_ir2, colors_ir1))
cmap_ir = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors_ir)
# --

# Pressure Level Wind Speed Colormap
# --
color_ws0070 = plt.cm.BuPu(np.linspace(0,0.6,149))
color_ws70120 = plt.cm.plasma(np.linspace(0.4,1,107))
color_wspl = np.vstack((color_ws0070, color_ws70120))
cmap_wspl = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_wspl)
# --    

# Vertical Velocity Colormap
# --
color_w1 = plt.cm.Greys_r(np.linspace(0,1,128))
color_w2 = plt.cm.inferno_r(np.linspace(0,1,128))
color_w = np.vstack((color_w1, color_w2))
cmap_w = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_w)
# --

# MUCAPE Colormap
# --
color_cp0050 = plt.cm.Greys(np.linspace(0.2,0.7,43))
color_cp50100 = plt.cm.Blues(np.linspace(0.2,0.7,42))
color_cp50300 = plt.cm.plasma_r(np.linspace(0,1,171))
color_cp = np.vstack((color_cp0050, color_cp50100, color_cp50300))
cmap_cp = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_cp)
# --     

# Snowdepth Colormap
# --
color_sn0002 = plt.cm.Greys(np.linspace(0.2,0.5,13))
color_sn0220 = plt.cm.cool(np.linspace(0,1,115))
color_sn2040 = plt.cm.RdPu(np.linspace(0.4,1,128))
color_sn = np.vstack((color_sn0002, color_sn0220, color_sn2040))
cmap_sn = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_sn)
# --     
# -----------------------------------------------------------------------------------

# PLOT DATA

# SURFACE
# ------------------
if Category == 'a':
    if Product == 'a':
        MEPSprod = '2m Temperature (°C) and 10m Winds (kt)'
        mesh = ax.pcolormesh(tmp2m.x, tmp2m.y, tmp2m -273.15, transform=tmp2m.metpy.cartopy_crs, cmap=cmap_t, vmin=-30, vmax=30)
        tc = ax.contour(tmp2m.metpy.x, tmp2m.metpy.y, tmp2m -273.15, levels=0, colors='white', linestyles='--', linewidths=0.6,
                        transform=tmp2m.metpy.cartopy_crs, antialiased=True)
        ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd2m[::res,::res].values)*1.94384,(vwnd2m[::res,::res].values)*1.94384,
                 transform=uwnd2m.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.2)

    if Product == 'b':
        MEPSprod = '2m Dewpoint (°C) and 10 Winds (kt)'
        mesh = ax.pcolormesh(td2m.x, td2m.y, td2m, transform=td2m.metpy.cartopy_crs, cmap=cmap_td, vmin=-30, vmax=30)
        ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd10m[::res,::res].values)*1.94384,(vwnd10m[::res,::res].values)*1.94384,
                 transform=uwnd10m.metpy.cartopy_crs,color='black',length=6, alpha=0.8, linewidth=0.3)

    if Product == 'c':
        MEPSprod = '10m Wind Speed (m/s, shaded), Wind Gusts (m/s, contour)'
        mesh = ax.pcolormesh(wndS.x, wndS.y, wndS, transform=wndS.metpy.cartopy_crs, cmap=cmap_ws, vmin=0, vmax=40)
        ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd2m[::res,::res].values)*1.94384,(vwnd2m[::res,::res].values)*1.94384,
                 transform=uwnd2m.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.4)
        wndGs = mpcalc.smooth_circular(wndG, 3, 2)
        wsp = ax.contour(wndGs.x, wndGs.y, wndGs, levels=(15,20,25,30,35,40), cmap=plt.cm.Reds, vmin=0, vmax=40, linewidths=1.5,
                   transform=wndS.metpy.cartopy_crs, antialiased=True)
        plt.clabel(wsp, fontsize=15, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True,
                    use_clabeltext=True)
    
    if Product == 'd': # Precipitation
        if Ptime =='a':
            MEPSprod = 'Precipitation Type Liquid water equivalent (Rain mm/h: Green, Snow mm/h: Blue)' 
            ax.pcolormesh(Graupel_Instant.x, Graupel_Instant.y, Graupel_Instant * 3600, transform=Graupel_Instant.metpy.cartopy_crs, cmap='Oranges',
                          norm=mcolors.LogNorm(vmin=0.1, vmax=10))
            ax.pcolormesh(Rain_Instant.x, Rain_Instant.y, Rain_Instant * 3600, transform=Rain_Instant.metpy.cartopy_crs, cmap='Greens',
                                 norm=mcolors.LogNorm(vmin=0.1, vmax=10))
            mesh = ax.pcolormesh(Snow_Instant.x, Snow_Instant.y, Snow_Instant * 3600, transform=Snow_Instant.metpy.cartopy_crs, cmap='BuPu',
                                 norm=mcolors.LogNorm(vmin=0.1, vmax=10))

            #ax.contourf(Graupel_Instant.x, Graupel_Instant.y, Graupel_Instant * 3600, levels=(np.arange(0,8,0.2)), transform=Graupel_Instant.metpy.cartopy_crs, cmap='Oranges',
             #                   norm=mcolors.LogNorm(vmin=0.2, vmax=10))
            #mesh2 = ax.contourf(Rain_Instant.x, Rain_Instant.y, Rain_Instant * 3600, levels=(np.arange(0,8,0.2)), transform=Rain_Instant.metpy.cartopy_crs, cmap='Greens',
             #                   norm=mcolors.LogNorm(vmin=0.2, vmax=10))
            #mesh = ax.contourf(Snow_Instant.x, Snow_Instant.y, Snow_Instant * 3600, levels=(np.arange(0,8,0.2)), transform=Snow_Instant.metpy.cartopy_crs,
             #                  norm=mcolors.LogNorm(vmin=0.2, vmax=10), cmap='BuPu')

            snowc = ax.contour(Snow_Instant.x, Snow_Instant.y, Snow_Instant * 3600, levels=(1,2,3,4,5), transform=Snow_Instant.metpy.cartopy_crs, colors='red')

            pres_smooth = mpcalc.smooth_circular(mslp, 3, 3)
            presc = ax.contour(pres_smooth.longitude, pres_smooth.latitude, pres_smooth / 100, transform=ccrs.PlateCarree(), levels=(np.arange(968,1040,4)), colors='k')
            ax.clabel(presc, fontsize=15, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True)
            blabel = [0.2,0.5,1,2,3,4,5,6,7,8,9,10]

            #ax.streamplot(uwnd2m.x, uwnd2m.y, uwnd2m, vwnd2m, transform=uwnd2m.metpy.cartopy_crs, density=1, linewidth=1, maxlength=1.5, color='k')
            ax.quiver(clons[::res, ::res], clats[::res,::res], uwnd2m[::res,::res].values, vwnd2m[::res,::res].values, transform=uwnd2m.metpy.cartopy_crs)
                      #scale=120)
        elif Ptime == 'b':
            MEPSprod = '1hr Precipitation mm (Shaded), Snowfall Total cm (contour)' 
            mesh = ax.contourf(precip_1hr.x, precip_1hr.y, precip_1hr, transform=precip_1hr.metpy.cartopy_crs, levels=(0.2,1,1.5,2,3,4,5,6,7,8,9,10), cmap='Greens', vmin=-4)
            snowc = ax.contour(snow_acm.x, snow_acm.y, snow_acm, transform = snow_integral.metpy.cartopy_crs, cmap='cool', levels=(2,5,10,15,20,25,30))
            ax.clabel(snowc, fontsize=8, colors='k', inline=1, inline_spacing=10, fmt='%i', rightside_up=True, use_clabeltext=True)
            pres_smooth = mpcalc.smooth_circular(mslp, 3, 3)
            presc = ax.contour(pres_smooth.longitude, pres_smooth.latitude, pres_smooth / 100, transform=ccrs.PlateCarree(), levels=(np.arange(968,1040,4)), colors='k')
            ax.clabel(presc, fontsize=15, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True)
    
    if Product == 'e': # Infrared Satellite
        MEPSprod = 'IR Simulated Satellite (°C)'
        cbar2 = 'true'
        mesh = ax.pcolormesh(ir.x, ir.y, ir, transform=ir.metpy.cartopy_crs, cmap=cmap_ir, vmin=-70, vmax=10)


# PRESSURE
# ------------------
elif Category == 'b':
    if Product == 'a':
        MEPSprod = str(plevel)+'hPa Winds (kt, shaded), Geopotential Height (dam, contours)'
        mesh = ax.contourf(swnd_pl.x, swnd_pl.y, swnd_pl*1.94384, transform=swnd_pl.metpy.cartopy_crs, 
                           levels=(10,20,30,40,50,60,70,80,90,100,110,120), cmap=cmap_wspl, vmin=0, vmax=120)
        ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd_pl[::res,::res].values)*1.94384,(vwnd_pl[::res,::res].values)*1.94384,
                 transform=uwnd_pl.metpy.cartopy_crs,color='black',length=length, alpha=0.8, linewidth=0.2)
        Geo_Pot = ax.contour(geop_pl.x, geop_pl.y, geop_pl / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')
        plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True)
    if Product == 'c':
        MEPSprod = str(plevel)+'hPa Relative Humidity (%, shaded)'
        mesh = ax.pcolormesh(rh_pl.x, rh_pl.y, rh_pl * 100, transform=rh_pl.metpy.cartopy_crs, cmap='BrBG', vmin=0, vmax=100, alpha=0.8)
        ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd_pl[::res,::res].values)*1.94384,(vwnd_pl[::res,::res].values)*1.94384,
                 transform=uwnd_pl.metpy.cartopy_crs,color='black',length=length, alpha=0.8, linewidth=0.2)  
        Geo_Pot = ax.contour(geop_pl.x, geop_pl.y, geop_pl / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')
        plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 
    if Product == 'd':
        MEPSprod = str(plevel)+'hPa Dew Point (°C, shaded)'
        mesh = ax.pcolormesh(td_pl.x, td_pl.y, td_pl, transform=td_pl.metpy.cartopy_crs, cmap=cmap_td, vmin=-30, vmax=30)
        ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd_pl[::res,::res].values)*1.94384,(vwnd_pl[::res,::res].values)*1.94384,
                 transform=uwnd_pl.metpy.cartopy_crs,color='black',length=length, alpha=0.8, linewidth=0.2)
        Geo_Pot = ax.contour(geop_pl.x, geop_pl.y, geop_pl / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')  
        plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 
    if Product == 'e':
        MEPSprod = str(plevel)+'hPa Verical Velocity'
        mesh = ax.pcolormesh(w_pl.x, w_pl.y, w_pl, transform = w_pl.metpy.cartopy_crs, cmap=cmap_w, vmin=-3, vmax=3)
       # ax.pcolormesh(w_pl.x, w_pl.y, wn_pl, transform = w_pl.metpy.cartopy_crs, cmap='gray', vmax=0, vmin=-3)
        Geo_Pot = ax.contour(geop_pl.x, geop_pl.y, geop_pl / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')  
        plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 

# T-STORMS
# ------------------
elif Category == 'c':
    if option == 'a':
        if Product == 'a':
            MEPSprod = 'MUCAPE (J/kg)'
            mesh = ax.pcolormesh(cape.x, cape.y, cape, transform = cape.metpy.cartopy_crs, cmap=cmap_cp, vmin=0, vmax=3000)
        if Product == 'b':
            MEPSprod = 'CIN (J/Kg)'
            mesh = ax.pcolormesh(cin.x, cin.y, cin, transform=cin.metpy.cartopy_crs, cmap='Greys', vmin=0, vmax=1000)
        if Product == 'c':
            MEPSprod = 'LCL (m)'
            mesh = ax.pcolormesh(lcl.x, lcl.y, lcl, transform=lcl.metpy.cartopy_crs, cmap='Greens_r', vmin=0, vmax=4000)
        if Product == 'd':
            MEPSprod = 'LFC (m)'
            mesh = ax.pcolormesh(lfc.x, lfc.y, lfc, transform=lfc.metpy.cartopy_crs, cmap='Greens_r', vmin=0, vmax=5000)
        if Product == 'e':
            MEPSprod = 'PBL Depth (m)'
            mesh = ax.pcolormesh(pbl.x, pbl.y, pbl, transform=pbl.metpy.cartopy_crs, cmap='Oranges', vmin=0, vmax=3000)
        if Product == 'f':
            MEPSprod = 'Precipitable Water'
            mesh = ax.pcolormesh(pw.x, pw.y, pw, transform=pw.metpy.cartopy_crs, cmap='BrBG', vmin=0, vmax=50)
            ax.contour(pw.x, pw.y, pw, transform=pw.metpy.cartopy_crs, colors='purple', levels=(12,25,75,100), linewidths=0.5)
        if Product == 'g':
            MEPSprod = 'Lightning Strikes'
            mesh = ax.pcolormesh(ltg.x, ltg.y, ltg, transform=ltg.metpy.cartopy_crs, cmap='RdPu')
        if Product == 'h':
            MEPSprod = 'Cloud Top Heights (m)'
            mesh = ax.pcolormesh(cld_top.x, cld_top.y, cld_top, transform=cld_top.metpy.cartopy_crs, cmap='inferno_r', vmin=0, vmax=12000)
        if Product == 'i':
            MEPSprod = 'Reflectivity (dBz)'
            #mesh = ax.contourf(dbz_r.x, dbz_r.y, dbz_r, transform=dbz_r.metpy.cartopy_crs, levels=(np.arange(0,72.5,2.5)), cmap=map_db, vmin=0, vmax=70)
            #ax.contourf(dbz_s.x, dbz_s.y, dbz_s, transform=dbz_s.metpy.cartopy_crs, levels=(np.arange(0,72.5,2.5)), cmap=map_db, vmin=0, vmax=70)
            mesh = ax.pcolormesh(dbz_r.x, dbz_r.y, dbz_r, transform=dbz_r.metpy.cartopy_crs, cmap=cmap_db, vmin=0, vmax=70)
        if Product == 'j':
            MEPSprod = '700 - 500mb Lapse Rate'
            lr57_s = mpcalc.smooth_circular(lr57, 4, 2)
            mesh = ax.contour(lr57.x, lr57.y, lr57_s, transform=lr57.metpy.cartopy_crs, levels=(np.arange(5,10.5,0.5)), cmap='Reds', vmin=0, vmax=10)
            plt.clabel(mesh, mesh.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 
    if option == 'b':
        if Product == 'a':
            MEPSprod = '2m Temperature and MUCAPE'
            cape_smooth = mpcalc.smooth_circular(cape, 3, 2)
            mesh = ax.pcolormesh(tmp2m.x, tmp2m.y, tmp2m -273.15, transform=tmp2m.metpy.cartopy_crs, cmap=cmap_t, vmin=-30, vmax=30, alpha=0.85)
            ax.contourf(cape.x, cape.y, cape_smooth, transform = cape.metpy.cartopy_crs, cmap='Reds', levels=(150,200,300,400,500,600,700), alpha=0.7)
            tc = ax.contour(tmp2m.metpy.x, tmp2m.metpy.y, tmp2m -273.15, levels=0, colors='grey', linestyles='--', linewidths=0.6,
                            transform=tmp2m.metpy.cartopy_crs, antialiased=True)
            #ax.streamplot(uwnd2m.x, uwnd2m.y, uwnd2m, vwnd2m, transform=uwnd2m.metpy.cartopy_crs, density=4)
            ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd2m[::res,::res].values)*1.94384,(vwnd2m[::res,::res].values)*1.94384,
                    transform=uwnd2m.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.2)
        if Product == 'c':
            MEPSprod = 'MUCAPE and 850mb Winds'
            mesh = ax.contourf(cape.x, cape.y, cape, transform = cape.metpy.cartopy_crs, cmap=cmap_cp, levels=(10,20,40,60,80,100,120,140,160,180,200))
            ax.barbs(clons[::res,::res],clats[::res,::res],(uwnd_85[::res,::res].values)*1.94384,(vwnd_85[::res,::res].values)*1.94384,
                    transform=uwnd_85.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.2)

# WINTER
# ------------------
elif Category == 'd':
    if Product == 'a':
        MEPSprod = 'Total Snowfall 10:1 (cm)'

        # Build Colormap
        mesh = ax.contourf(snow_acm.x, snow_acm.y, snow_acm, transform=snow_acm.metpy.cartopy_crs, 
                           levels=(0.2,0.5,1,1.5,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40), 
                           cmap=cmap_sn, vmin=0, vmax=40)
        blabel = [0.2,1,2,5,10,15,20,25,30,35,40]
        ax.contour(snow_acm.x, snow_acm.y, snow_acm, transform=snow_acm.metpy.cartopy_crs, levels=(5,10,15,20,25,30), colors='red', linewidths=0.6)
    if Product == 'e':
        MEPSprod = 'Precipitation Type'
        mesh = ax.pcolormesh(x_crd, y_crd, ptype, transform=ptype.metpy.cartopy_crs, cmap='tab20c' )


#----------------------------------------#


#----------------------------------------#
# Plot Ad-ons

# Color Bar
if 'blabel' in locals(): 
     plt.colorbar(mesh, orientation='horizontal',fraction=fraction1,pad=0.0,aspect=60,extendrect=True, ticks=blabel).ax.set_xticklabels(blabel)
     del blabel
else:
    plt.colorbar(mesh, orientation='horizontal',fraction=fraction1,pad=0.0,aspect=60,extendrect=True)

# Title
plt.title('MEPS || ' +str(MEPSprod), size=10, loc='left')
plt.title('MEPS '+((FF00.dt.strftime('%HZ')).values)+' | Valid '+((valid_time.dt.strftime('%Y-%m-%d %HZ')).values), size=10, loc='right')

# Water Mark
plt.text(xa, ya, '@FIN_storm', size=25, alpha=0.3,weight='bold', 
     horizontalalignment='center',
     verticalalignment='center',
     transform = ax.transAxes)
#----------------------------------------#